### setup

In [ ]:
library(EWCE) # Versions prior to 1.5.5 did not compute controlled results properly
library(orthogene)
library(Seurat)

In [ ]:
wd <- '~/codebases/MacBrainDev/'
data.dir <- 'data/'
setwd(wd)


n.cores <-  as.numeric(Sys.getenv('SLURM_CPUS_PER_TASK'))
n.cores <- if (!is.na(n.cores) & n.cores > 1) n.cores else parallel::detectCores()
n.cores

options(future.globals.maxSize= 5*1024^3)
future::plan(strategy = "multicore", workers = n.cores)

disease_lists <- readRDS(glue::glue('{data.dir}Disease_genes/all_diseases_list.rds'))

# For markers only
downsample_ncells_per_subtype <- Inf

# Define filenames

base.name <- 'All.MNN.v1.org.fct'

indata.fname <- paste0(data.dir, base.name, '.rds')
# ctd object
ctd.fname <- paste0(data.dir, 'ewce_ctd.', base.name,
                    '.ds', downsample_ncells_per_subtype, '.rds')
# markers
markers.fname <- paste0(data.dir, 'ewce_downsampled_markers.',
                        base.name,  '.ds', 
                        downsample_ncells_per_subtype,'.rds')
# sct object
sct.fname <- paste0(data.dir, 'ewce_sct.', base.name, '.rds')
# sct normalized object
sct.normed.fname <- paste0(data.dir, 'ewce_sct_normed_markers_orthos.', base.name, 
                           '.ds', downsample_ncells_per_subtype, '.rds')

# Define parameters
# for ewce
reps <- 20000 #  >10000
verbose <- T

# ewce
dis.res.folder <- paste0(data.dir, 'EWCE_diseases.ds', downsample_ncells_per_subtype, '/')
dir.create(dis.res.folder, showWarnings = F)
EWCE_results.fname <- paste0('EWCE_results.', reps, '_reps.ds', downsample_ncells_per_subtype, '.csv')


In [ ]:
# Load data
indata <- readRDS(indata.fname)
# Print summary
indata

Idents(indata) <- 'subtype'

### Find All Marker genes

In [ ]:
normalizePath(markers.fname)

In [ ]:
markers.fname

# Check if run
if (!file.exists(markers.fname) & !file.exists(ctd.fname)){
    
    print('Computing Markers')
    
    Idents(indata) <- 'subtype'
    
    all.markers <- FindAllMarkers(
      # ds.seurat,
      indata,
      assay = 'RNA',
      features = NULL,
      logfc.threshold = 0.25,
      test.use = "wilcox",
      slot = "data",
      min.pct = 0.1,
      min.diff.pct = -Inf,
      node = NULL,
      verbose = TRUE,
      only.pos = FALSE,
      max.cells.per.ident = downsample_ncells_per_subtype,
      random.seed = 1,
      latent.vars = NULL,
      min.cells.feature = 3,
      min.cells.group = 3,
      pseudocount.use = 1,
      mean.fxn = NULL,
      fc.name = NULL,
      base = 2,
      return.thresh = 0.01,
      densify = FALSE,
    )
    
    saveRDS(all.markers, markers.fname)
    
} else {
    all.markers <- readRDS(markers.fname)
}

# ADJUSTED P-VALUE < 0.05
length(unique(dplyr::filter(all.markers, p_val_adj < 0.05)$gene))

# ADJUSTED P-VALUE < 0.05 & |AVG log2 FC| > 1 
length(unique(dplyr::filter(all.markers, p_val_adj < 0.05, abs(avg_log2FC) > 1)$gene)) 

# ADJUSTED P-VALUE < 0.05 & |AVG log2 FC| > 2
length(unique(dplyr::filter(all.markers, p_val_adj < 0.05, abs(avg_log2FC) > 2)$gene)) 

significant.markers <- unique(dplyr::filter(all.markers, p_val_adj < 0.05)$gene)

### SCT Normalization

In [ ]:
# Lower number of cores to reduce memory usagefuture::plan(strategy = "multicore", workers = min(40, n.cores))

file.exists(sct.normed.fname)
ctd.fname
file.exists(ctd.fname)
# If CTD is already computed, we don't need anything here
if (file.exists(sct.normed.fname) & !file.exists(ctd.fname)){
    exp.normed <- readRDS(sct.normed.fname)
    
} else if (!file.exists(ctd.fname)){
    
    # If we already have the sct-data, we can load it, otherwise we run it
    if (file.exists(sct.fname)){
        sct <- readRDS(sct.fname)
    } else {
        requireNamespace("sctransform")

        raw.expression <- indata[['RNA']]@counts
        # First, perform sctransform of data
        sct <- sctransform::vst(umi = raw.expression,
                                method="glmGamPoi", 
                                return_cell_attr = TRUE, 
                                n_genes = 3000,
                                verbosity = 2)
        saveRDS(sct, sct.fname)
    }
    
    ## Get orthologues data
    # Check number of genes in SCT-data and in significant markers
    print(paste('Number of genes before SCT:', nrow(indata)))
    sct.genes <- rownames(sct$y)
    print(paste('Number of genes after SCT:', length(sct.genes)))
    print(paste('Number of marker genes:', length(significant.markers)))
    # Intersect lists
    markers.sct.genes <- sct.genes[sct.genes %in% significant.markers]
    print(paste('Number of genes after SCT in markers:', length(markers.sct.genes)))
    # Convert macaque to human genes
    orthos <- convert_orthologs(markers.sct.genes,input_species = 'macaque', verbose = F)
    print(paste('Number of matching orthologs:', nrow(orthos)))

    ## Correct orthologues data
    # Get subset of data
    ortho.raw <- indata[['RNA']]@counts[orthos$input_gene,]

    # Correct subset data using full model
    ortho.sct <- sctransform::correct_counts(x = sct, umi = ortho.raw, verbosity = 2)
    # Rename genes in object
    rownames(ortho.sct) <- rownames(orthos)
    # Normalize gene expression
    exp.normed <- Matrix::t(Matrix::t(ortho.sct) * (1/Matrix::colSums(ortho.sct)))
    
    saveRDS(exp.normed, sct.normed.fname)
    
}

### CellTypeData object

In [ ]:
# increase number of cores for parallel computing
future::plan(strategy = "multicore", workers = min(80, n.cores))

# Define annotation levels
annot.levels <- as.list(indata@meta.data[,c('subclass','subtype')])

if (file.exists(ctd.fname)){

    ctd <- ctd.fname

} else {

    ctd <- generate_celltype_data(
    
    exp=exp.normed,
    annotLevels=annot.levels,
    groupName='MacaqueDev',
    
    input_species='hsapiens',
    output_species='hsapiens',
    convert_orths=F,
    
    specificity_quantiles=T,
    dendrograms=T,
    
    file_prefix=paste0('ewce_ctd.',base.name),
    force_new_file=T,
    
    no_cores=n.cores,
    as_sparse = TRUE,
    as_DelayedArray = FALSE,
    verbose=T)
    
    file.copy(ctd, ctd.fname)
}

### Perform analysis

In [ ]:
print(Sys.time())

In [ ]:
sapply(disease_lists, length)

if (is.character(ctd)){load(ctd)}

# Start buffer of results
dis_results <- list()

max_tries <- 10
geneSizeControl <- F

# Iterate lists of diseases
for (dis in names(disease_lists)){
    
    # get genset
    gset <- disease_lists[[dis]]

    # Decide which level of annotation to analyse, we do both
    for (level in c(1:2)){

        # Start summary
print(paste(which(names(disease_lists)==dis), length(disease_lists), sep='/'))
        print(paste('### Disease:', dis))
        print(paste('### GC&Length control:', geneSizeControl))
        print(paste('### Level:', level))
        
        # define a name of the run and filename
        tag <- paste(dis, level, geneSizeControl) 
        dis_results.fname <- paste0(dis.res.folder, 'EWCE_results.', reps, '_reps.', tag, '.csv')

        # If results were computed, load them
        if (file.exists(dis_results.fname)){
            print('Loading results')
            dis_results[[tag]] <- read.csv(dis_results.fname, row.names=1)
        # If not, try to compute it upto max.tries
        } else {
            tries <- 0
            while (!tag %in% names(dis_results) & tries < max_tries){
                tryCatch(
                    {
                        tries <- tries +1
                        res <- bootstrap_enrichment_test(

                            sct_data = ctd,

                            hits = gset,
                            bg = NULL,

                            genelistSpecies = 'human',
                            sctSpecies = 'human',
                            output_species = "human",

                            reps = reps,

                            annotLevel = level,
                            geneSizeControl = geneSizeControl,
                            controlledCT = NULL,
                            mtc_method = "BH",

                            no_cores = n.cores,
                            sort_results = TRUE,
                            verbose = verbose)


                        res$results$hit.cells <- res$hit.cells[rownames(res$results)]
                        res$results$geneSizeControl <- geneSizeControl
                        
                        # Save, export and delete
                        dis_results[[tag]] <- data.frame(res$results, list=dis)
                        write.csv(dis_results[[tag]], dis_results.fname)
                        rm(res)
                    },
                    error=function(cond) {
                        print(as.character(cond))

                        print(tag)
                        if (tries > max_tries){
                            print('COULD NOT COMPUTE')
                        } else {
                            print('Repeating')
                        }
                        # Choose a return value in case of error
                        return()
                    })
                gc()
            }
        }
        print('next...')
    }
}

# Combine results
results <- do.call('rbind', dis_results)
rownames(results) <- 1:nrow(results)
results

# Export results
write.csv(results, file=EWCE_results.fname)

In [ ]:
print(Sys.time())

In [ ]:
sessionInfo()